In [ ]:
import pandas as pd
import numpy as np

import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report, f1_score

In [43]:
df = pd.read_csv("train.csv")
df = df.drop(columns="id")
df.head()

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [44]:
# Keep multi-label targets; do not collapse to a single label
label_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
df = df[['comment_text'] + label_names]

# Quick sanity check: positive counts per label
df[label_names].sum()

toxic            15294
severe_toxic      1595
obscene           8449
threat             478
insult            7877
identity_hate     1405
dtype: int64

In [45]:
for i in df.columns:
    print(df[i].value_counts())

comment_text
Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

Text Preprocessing

In [46]:
df["comment_text"] = df["comment_text"].str.lower()
df.head()

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,explanation\nwhy the edits made under my usern...,0,0,0,0,0,0
1,d'aww! he matches this background colour i'm s...,0,0,0,0,0,0
2,"hey man, i'm really not trying to edit war. it...",0,0,0,0,0,0
3,"""\nmore\ni can't make any real suggestions on ...",0,0,0,0,0,0
4,"you, sir, are my hero. any chance you remember...",0,0,0,0,0,0


Data Cleaning

In [47]:
import re
import nltk
from nltk.corpus import stopwords
from bs4 import BeautifulSoup

import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/abhishek/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [48]:
#Removing special characters 
df["comment_text"]=df["comment_text"].apply(lambda x: re.sub("[^a-z A-Z 0-9]", " ", x))

#Removing all stopwords
df["comment_text"]=df["comment_text"].apply(lambda x: " ".join([word for word in x.split() if word not in (stopwords.words('english'))]))

#Remove all urls 
df["comment_text"]=df["comment_text"].apply(lambda x: re.sub(r"(http\S+|www\S+|https\S+)", '', x))

#Remove any additional spaces 
df["comment_text"]=df["comment_text"].apply(lambda x: " ".join(x.split()))


In [49]:
df.head()

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,explanation edits made username hardcore metal...,0,0,0,0,0,0
1,aww matches background colour seemingly stuck ...,0,0,0,0,0,0
2,hey man really trying edit war guy constantly ...,0,0,0,0,0,0
3,make real suggestions improvement wondered sec...,0,0,0,0,0,0
4,sir hero chance remember page,0,0,0,0,0,0


Lemmatization

In [50]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/abhishek/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [51]:
lemmatizer = WordNetLemmatizer()

def lemmatize_text(text: str) -> str:
    # Lemmatize token-wise (no POS tags for simplicity)
    tokens = text.split()
    return " ".join(lemmatizer.lemmatize(w) for w in tokens)

# Optional: This can be slow; you may skip if desired
df["comment_text"] = df["comment_text"].apply(lemmatize_text)

In [52]:
df.head()

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,explanation edits made username hardcore metal...,0,0,0,0,0,0
1,aww match background colour seemingly stuck th...,0,0,0,0,0,0
2,hey man really trying edit war guy constantly ...,0,0,0,0,0,0
3,make real suggestion improvement wondered sect...,0,0,0,0,0,0
4,sir hero chance remember page,0,0,0,0,0,0


Train Test Split

In [53]:
X = df["comment_text"]
y = df[label_names]  # multi-label

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=None  # stratify for multilabel is non-trivial
)

Text to Vector 

In [54]:
tfidf = TfidfVectorizer(
    ngram_range=(1, 2),
    min_df=2,
    max_df=0.95,
    sublinear_tf=True
)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

In [55]:
ovr = OneVsRestClassifier(
    LogisticRegression(max_iter=1000, class_weight='balanced', solver='liblinear')
)
ovr.fit(X_train_tfidf, y_train)

,estimator,LogisticRegre...r='liblinear')
,n_jobs,None
,verbose,0
,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,'balanced'
,random_state,None


In [56]:
# Use probabilities and a 0.5 threshold (you can tune per-label thresholds)
y_proba = ovr.predict_proba(X_test_tfidf)
y_pred = (y_proba >= 0.5).astype(int)

print(classification_report(y_test, y_pred, target_names=label_names, zero_division=0))
print("Macro F1:", f1_score(y_test, y_pred, average='macro'))

               precision    recall  f1-score   support

        toxic       0.69      0.84      0.75      3056
 severe_toxic       0.29      0.81      0.43       321
      obscene       0.73      0.87      0.79      1715
       threat       0.26      0.73      0.38        74
       insult       0.60      0.85      0.70      1614
identity_hate       0.28      0.73      0.41       294

    micro avg       0.60      0.84      0.70      7074
    macro avg       0.47      0.81      0.58      7074
 weighted avg       0.64      0.84      0.72      7074
  samples avg       0.06      0.08      0.07      7074

Macro F1: 0.5777516171770049


In [68]:
def predict_toxic(texts, threshold=0.5):
    """
    texts: list[str]
    returns: list of dicts with per-label probabilities, binary preds, and toxic_any flag
    """
    Xv = tfidf.transform(texts)
    proba = ovr.predict_proba(Xv)
    results = []
    for p in proba:
        probs = {name: float(p[i]) for i, name in enumerate(label_names)}
        preds = {name: int(p[i] >= threshold) for i, name in enumerate(label_names)}
        results.append({
            "prob": probs,
            "pred": preds,
            "toxic_any": int(any(v == 1 for v in preds.values()))
        })
    return results

import numpy as np

def predict_binary_label(texts, threshold=0.5):
    Xv = tfidf.transform(texts)
    proba = ovr.predict_proba(Xv)
    labels = []
    for p in proba:
        toxic_any = int(any(p[i] >= threshold for i in range(len(label_names))))
        labels.append("toxic" if toxic_any else "clean")
    return labels

# Example
predict_binary_label(["you ugly woman"])

['toxic']